In [194]:
import tqdm
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random
from icecream import ic


In [207]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [196]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy Algorithm

In [197]:
def forms_loop(tsp, start, end):
    visited = set()
    current = start
    while current != -1:
        if current == end:
            return True
        visited.add(current)
        next_city = -1
        for s, e in tsp:
            if s == current and e not in visited:
                next_city = e
                break
            elif e == current and s not in visited:
                next_city = s
                break
        current = next_city
    return False

In [204]:
tsp = []
city_count = {i: 0 for i in range(len(CITIES))}

while len(tsp) < len(CITIES) - 1:
    min_dist = float('inf')
    best_pair = None
    
    # Find the shortest edge that doesn't violate the TSP rules
    for i in range(len(CITIES)):
        for j in range(i + 1, len(CITIES)):
            if city_count[i] < 2 and city_count[j] < 2 and not forms_loop(tsp, i, j):
                dist = DIST_MATRIX[i, j]
                if dist < min_dist:
                    min_dist = dist
                    best_pair = (i, j)
    
    # Add the selected edge to the path
    if best_pair:
        i, j = best_pair
        tsp.append((i, j))
        city_count[i] += 1
        city_count[j] += 1

# Add the last edge to complete the loop
for i in range(len(CITIES)):
    for j in range(i + 1, len(CITIES)):
        if city_count[i] < 2 and city_count[j] < 2:
            tsp.append((i, j))
            break

# Convert tsp edges to ordered path
path = [tsp[0][0], tsp[0][1]]
while len(path) < len(CITIES):
    current_city = path[-1]
    # Find the next city in tsp edges that connects to the current city
    for s, e in tsp:
        if s == current_city and e not in path:
            path.append(e)
            break
        elif e == current_city and s not in path:
            path.append(s)
            break

# Complete the loop by returning to the starting city
path.append(path[0])

print(tsp)
print("TSP Path", path)
print(tsp_cost(path), len(path)-1)

[(176, 180), (76, 201), (266, 287), (8, 104), (83, 190), (3, 12), (96, 239), (8, 209), (67, 262), (36, 109), (180, 220), (12, 313), (28, 233), (65, 265), (69, 220), (106, 265), (212, 259), (13, 317), (239, 258), (136, 190), (4, 225), (131, 159), (65, 127), (28, 37), (225, 240), (96, 236), (316, 319), (284, 312), (87, 282), (186, 294), (87, 216), (138, 243), (40, 154), (107, 240), (109, 217), (208, 236), (140, 238), (43, 121), (108, 172), (4, 168), (120, 176), (17, 188), (56, 102), (76, 78), (297, 317), (44, 108), (24, 241), (206, 211), (85, 315), (95, 291), (98, 227), (39, 203), (119, 216), (174, 193), (136, 195), (71, 75), (228, 307), (256, 319), (237, 277), (200, 288), (104, 106), (30, 244), (13, 145), (98, 124), (95, 140), (24, 202), (158, 201), (92, 305), (146, 215), (179, 252), (170, 186), (69, 124), (62, 227), (131, 301), (79, 167), (27, 171), (138, 208), (44, 295), (226, 315), (78, 85), (75, 312), (23, 238), (278, 279), (171, 187), (120, 175), (274, 298), (101, 287), (150, 282),

## Genetic Algorithm

In [199]:
def fitness(route):
    distance = sum(DIST_MATRIX[route[i], route[i+1]] for i in range(len(route) - 1))
    distance += DIST_MATRIX[route[-1], route[0]]
    return distance

# Initialize population
def initial_population(pop_size):
    population = []
    for _ in range(pop_size):
        route = list(range(len(CITIES)))
        random.shuffle(route)
        population.append(route)
    return population

# Selection - tournament selection
def selection(population, fitnesses):
    selected = random.choices(population, weights=fitnesses, k=2)
    return selected[0], selected[1]

# Crossover - ordered crossover
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    pointer = end
    for city in parent2:
        if city not in child:
            if pointer >= len(parent1):
                pointer = 0
            child[pointer] = city
            pointer += 1
    return child

# Mutation - swap mutation
def mutate(mutation_rate,route):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]

In [208]:
POP_SIZE =500    # Population size
GENS = 500           # Number of generations
MUTATION_RATE = 0.05 # Probability of mutation
ELITISM = 0.05


population = initial_population(POP_SIZE)

for gen in tqdm.tqdm(range(GENS)):
    # Evaluate fitness
    fitnesses = [1 / fitness(route) for route in population] # Higher fitness is better

    # Elitism - carry over the best routes
    elite_count = int(ELITISM * POP_SIZE)
    sorted_population = [route for _, route in sorted(zip(fitnesses, population), reverse=True)]
    new_population = sorted_population[:elite_count]

    # Create new population
    while len(new_population) < POP_SIZE:
        parent1, parent2 = selection(population, fitnesses)
        child = crossover(parent1, parent2)
        mutate(MUTATION_RATE,child)
        new_population.append(child)

    population = new_population

    # Report the best route of the generation
    best_route = min(population, key=fitness)

# Output the best route after evolution
best_route = min(population, key=fitness)
best_route= best_route + [best_route[0]]
print("Best TSP Route (by indices):", best_route)
print("Total Distance:", tsp_cost(best_route))

100%|██████████| 500/500 [14:07<00:00,  1.69s/it]

Best TSP Route (by indices): [98, 681, 127, 304, 558, 698, 678, 222, 420, 135, 209, 535, 342, 41, 6, 403, 243, 234, 61, 718, 469, 275, 156, 691, 501, 643, 344, 509, 489, 241, 533, 600, 218, 470, 84, 422, 561, 543, 290, 530, 504, 707, 473, 205, 134, 31, 159, 245, 22, 452, 173, 300, 210, 552, 3, 192, 505, 356, 481, 570, 126, 151, 62, 80, 323, 404, 119, 619, 392, 293, 97, 355, 712, 366, 232, 667, 522, 394, 429, 683, 340, 115, 627, 53, 189, 574, 221, 272, 314, 215, 654, 496, 466, 602, 426, 280, 525, 428, 568, 656, 109, 369, 348, 541, 266, 233, 696, 19, 597, 461, 130, 296, 284, 1, 116, 669, 615, 21, 279, 352, 255, 569, 487, 108, 198, 5, 640, 117, 29, 405, 406, 665, 129, 303, 414, 554, 120, 11, 184, 153, 39, 551, 143, 52, 40, 450, 329, 76, 261, 42, 666, 79, 649, 325, 388, 213, 716, 225, 82, 548, 624, 364, 399, 92, 528, 9, 367, 17, 562, 464, 190, 49, 653, 621, 295, 370, 372, 659, 462, 625, 310, 411, 717, 140, 440, 454, 206, 471, 395, 81, 168, 705, 536, 424, 690, 546, 529, 43, 85, 141, 472, 59